In [1]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
#sns.set_style("darkgrid")
#sns.set_context("poster")
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
%matplotlib inline
import sys
import os
from scipy.optimize import newton, minimize, fsolve
import numpy as np
import copy
import re

import matplotlib.colors as mcolors
palette = list(mcolors.TABLEAU_COLORS.keys())
palette.extend(['salmon', 'teal','yellowgreen'])

In [2]:
def loss_func_c1(t, y):
    return((np.abs(y - t) / t) * 100)
def loss_func_l1(t, y):
    return(np.abs(y - t))  # l1
def loss_func_l2(t, y):
    return((y - t)**2)     # l2


def lin_fit(x, beta):
    return(x*beta)

def obj_lin_fit(loss_func, amt_uM, rsp_ratio, p):
    beta = p[0]
    y = lin_fit(rsp_ratio, beta)
    loss = sum(loss_func(amt_uM, y))
    return(loss)

def lin_fit_off(x, alpha, beta):
    return(alpha + x*beta)

def obj_lin_fit_off(loss_func, amt_uM, rsp_ratio, p):
    alpha = p[0]
    beta = p[1]
    y = lin_fit_off(rsp_ratio, alpha, beta)
    loss = sum(loss_func(amt_uM, y))
    return(loss)


def pol2d_fit(x, beta, gamma):
    return(x**2*beta + x*gamma)

def obj_pol2d_fit(loss_func, amt_uM, rsp_ratio, p):
    beta = p[0]
    gamma = p[1]
    y = pol2d_fit(rsp_ratio, beta, gamma)
    loss = sum(loss_func(amt_uM, y))
    return(loss)

def pol2d_fit_off(x, alpha, beta, gamma):
    return(alpha + x**2*beta + x*gamma)

def obj_pol2d_fit_off(loss_func, amt_uM, rsp_ratio, p):
    alpha = p[0]
    beta = p[1]
    gamma = p[2]
    y = pol2d_fit_off(rsp_ratio, alpha, beta, gamma)
    loss = sum(loss_func(amt_uM, y))
    return(loss)


def pow_fit(x, beta, gamma):
    return(beta*x**gamma)

def obj_pow_fit(loss_func, amt_uM, rsp_ratio, p):
    beta = p[0]
    gamma = p[1]
    y = pow_fit(rsp_ratio, beta, gamma)
    loss = sum(loss_func(amt_uM, y))
    return(loss)

def pow_fit_off(x, alpha, beta, gamma):
    return(alpha + beta*x**gamma)

def obj_pow_fit_off(loss_func, amt_uM, rsp_ratio, p):
    alpha = p[0]
    beta = p[1]
    gamma = p[2]
    y = pow_fit_off(rsp_ratio, alpha, beta, gamma)
    loss = sum(loss_func(amt_uM, y))
    return(loss)


def pick_best_fun(t, x):
    bnds = ((0, None),)
    def fun_lin(p): return(obj_lin_fit(loss_func_c1, t, x, p))
    p_lin = minimize(fun_lin, (1), method='L-BFGS-B', bounds=bnds)

    bnds = ((None, None), (0, None))
    def fun_pol2d(p): return(obj_pol2d_fit(loss_func_c1, t, x, p))
    p_pol2d = minimize(fun_pol2d, (1, 1), method='L-BFGS-B', bounds=bnds)

    bnds = ((0, None), (0, None))
    def fun_pow(p): return(obj_pow_fit(loss_func_c1, t, x, p))
    p_pow = minimize(fun_pow, (0.4, 1), method='L-BFGS-B', bounds=bnds)

    loss_list = [p_lin.fun, p_pol2d.fun, p_pow.fun]
    func_list = [lambda x: lin_fit(x, p_lin.x[0]), lambda x: pol2d_fit(x, p_pol2d.x[0], p_pol2d.x[1]), lambda x: pow_fit(x, p_pow.x[0], p_pow.x[1])]
    min_idx = loss_list.index(min(loss_list))
    best_fun = func_list[min_idx]

    return(best_fun)

def pick_best_fun_l1(t, x):
    bnds = ((0, None),)
    def fun_lin(p): return(obj_lin_fit(loss_func_l1, t, x, p))
    p_lin = minimize(fun_lin, (1), method='L-BFGS-B', bounds=bnds)

    bnds = ((None, None), (0, None))
    def fun_pol2d(p): return(obj_pol2d_fit(loss_func_l1, t, x, p))
    p_pol2d = minimize(fun_pol2d, (1, 1), method='L-BFGS-B', bounds=bnds)

    bnds = ((0, None), (0, None))
    def fun_pow(p): return(obj_pow_fit(loss_func_l1, t, x, p))
    p_pow = minimize(fun_pow, (0.4, 1), method='L-BFGS-B', bounds=bnds)

    loss_list = [p_lin.fun, p_pol2d.fun, p_pow.fun]
    func_list = [lambda x: lin_fit(x, p_lin.x[0]), lambda x: pol2d_fit(x, p_pol2d.x[0], p_pol2d.x[1]), lambda x: pow_fit(x, p_pow.x[0], p_pow.x[1])]
    min_idx = loss_list.index(min(loss_list))
    best_fun = func_list[min_idx]

    return(best_fun)

def pick_best_fun_l2(t, x):
    bnds = ((0, None),)
    def fun_lin(p): return(obj_lin_fit(loss_func_l2, t, x, p))
    p_lin = minimize(fun_lin, (1), method='L-BFGS-B', bounds=bnds)

    bnds = ((None, None), (0, None))
    def fun_pol2d(p): return(obj_pol2d_fit(loss_func_l2, t, x, p))
    p_pol2d = minimize(fun_pol2d, (1, 1), method='L-BFGS-B', bounds=bnds)

    bnds = ((0, None), (0, None))
    def fun_pow(p): return(obj_pow_fit(loss_func_l2, t, x, p))
    p_pow = minimize(fun_pow, (0.4, 1), method='L-BFGS-B', bounds=bnds)

    loss_list = [p_lin.fun, p_pol2d.fun, p_pow.fun]
    func_list = [lambda x: lin_fit(x, p_lin.x[0]), lambda x: pol2d_fit(x, p_pol2d.x[0], p_pol2d.x[1]), lambda x: pow_fit(x, p_pow.x[0], p_pow.x[1])]
    min_idx = loss_list.index(min(loss_list))
    best_fun = func_list[min_idx]

    return(best_fun)



def pick_best_fun_off(t, x):
    bnds = ((None, None), (0, None))
    def fun_lin(p): return(obj_lin_fit_off(loss_func_c1, t, x, p))
    p_lin = minimize(fun_lin, (0, 1), method='L-BFGS-B', bounds=bnds)

    bnds = ((None, None), (None, None), (0, None))
    def fun_pol2d(p): return(obj_pol2d_fit_off(loss_func_c1, t, x, p))
    p_pol2d = minimize(fun_pol2d, (0, 1, 1), method='L-BFGS-B', bounds=bnds)

    bnds = ((None, None), (0, None), (0, None))
    def fun_pow(p): return(obj_pow_fit_off(loss_func_c1, t, x, p))
    p_pow = minimize(fun_pow, (0, 0.4, 1), method='L-BFGS-B', bounds=bnds)

    loss_list = [p_lin.fun, p_pol2d.fun, p_pow.fun]
    func_list = [lambda x: lin_fit_off(x, p_lin.x[0], p_lin.x[1]), lambda x: pol2d_fit_off(x, p_pol2d.x[0], p_pol2d.x[1], p_pol2d.x[2]), lambda x: pow_fit_off(x, p_pow.x[0], p_pow.x[1], p_pow.x[2])]
    min_idx = loss_list.index(min(loss_list))
    best_fun = func_list[min_idx]

    return(best_fun)


def pick_best_fun_off_l1(t, x):
    bnds = ((None, None), (0, None))
    def fun_lin(p): return(obj_lin_fit_off(loss_func_l1, t, x, p))
    p_lin = minimize(fun_lin, (0, 1), method='L-BFGS-B', bounds=bnds)

    bnds = ((None, None), (None, None), (0, None))
    def fun_pol2d(p): return(obj_pol2d_fit_off(loss_func_l1, t, x, p))
    p_pol2d = minimize(fun_pol2d, (0, 1, 1), method='L-BFGS-B', bounds=bnds)

    bnds = ((None, None), (0, None), (0, None))
    def fun_pow(p): return(obj_pow_fit_off(loss_func_l1, t, x, p))
    p_pow = minimize(fun_pow, (0, 0.4, 1), method='L-BFGS-B', bounds=bnds)

    loss_list = [p_lin.fun, p_pol2d.fun, p_pow.fun]
    func_list = [lambda x: lin_fit_off(x, p_lin.x[0], p_lin.x[1]), lambda x: pol2d_fit_off(x, p_pol2d.x[0], p_pol2d.x[1], p_pol2d.x[2]), lambda x: pow_fit_off(x, p_pow.x[0], p_pow.x[1], p_pow.x[2])]
    min_idx = loss_list.index(min(loss_list))
    best_fun = func_list[min_idx]

    return(best_fun)



def pick_lin_fun(t, x, off):
    if off is True:
        bnds = ((None, None), (0, None))
        def fun_lin(p): return(obj_lin_fit_off(loss_func_l1, t, x, p))
        p_lin = minimize(fun_lin, (0, 1), method='L-BFGS-B', bounds=bnds)
        return(lambda x: lin_fit_off(x, p_lin.x[0], p_lin.x[1]))
    elif off is False:
        bnds = ((0, None),)
        def fun_lin(p): return(obj_lin_fit(loss_func_l1, t, x, p))
        p_lin = minimize(fun_lin, (1), method='L-BFGS-B', bounds=bnds)
        return(lambda x: lin_fit(x, p_lin.x[0]))
    else:
        raise Exception('Unrecognized off: {}'.format(off))

In [3]:
def make_piece_wise_fit(amt_uM_range, rsp_ratio_range, df, debug=True):
    '''
    This function returns a piecewise function y = f(x)
    x = response ratio
    y = imputed sample concentration
    '''
    piece_wise_fit = dict()
    smo_frac = 0.4 # linear smoothening, fraction +/- response ratio

    ### Extrapolate above highest response ratio ###
    ub = rsp_ratio_range[0]
    lb = rsp_ratio_range[1] + smo_frac*rsp_ratio_range[1]
    key_extra_up = (ub, lb)

    mask = (amt_uM == amt_uM_range[1]) | (amt_uM == amt_uM_range[2]) | (amt_uM == amt_uM_range[3]) | (amt_uM == amt_uM_range[4]) | (amt_uM == amt_uM_range[5])
    amt_uM_tmp = amt_uM[mask]
    rsp_ratio_tmp = rsp_ratio[mask]

    # Restrict the extrapolation to linear fit:
    # piece_wise_fit[key_extra_up] = pick_best_fun(amt_uM_tmp, rsp_ratio_tmp)
    piece_wise_fit[key_extra_up] = pick_lin_fun(amt_uM_tmp, rsp_ratio_tmp, True)


    ### Interpolate between highest and lowest response ratio ###
    ub = rsp_ratio_range[1] - smo_frac*rsp_ratio_range[1]
    lb = rsp_ratio_range[len(amt_uM_range)-2] + smo_frac*rsp_ratio_range[len(amt_uM_range)-2]
    key_intra = (ub, lb)
    piece_wise_fit[key_intra] = pick_best_fun_off(amt_uM, rsp_ratio)


    ### Smoothen between the upper extrapolation and the intrapolation ###
    ub = rsp_ratio_range[1] + smo_frac*rsp_ratio_range[1]
    lb = rsp_ratio_range[1] - smo_frac*rsp_ratio_range[1]
    key_between_intra_up = (ub, lb)
    
    y_upper = piece_wise_fit[key_extra_up](ub)
    y_lower = piece_wise_fit[key_intra](lb)

    beta_upper = (y_upper - y_lower) / (ub - lb)
    if beta_upper < 0:
        print('Upper smoothening gave negative slope!!! Maybe adjust smo_frac.')
    alpha_upper = y_upper - beta_upper*ub
    piece_wise_fit[key_between_intra_up] = lambda x: alpha_upper + beta_upper*x


    ### Extrapolate between 0 and lowest response ratio ###
    ub = rsp_ratio_range[len(amt_uM_range)-2] - smo_frac*rsp_ratio_range[len(amt_uM_range)-2]
    lb = rsp_ratio_range[len(amt_uM_range)-1]
    key_extra_low = (ub, lb)

    i = len(amt_uM_range)-2
    # Extrapolate from the 5 lowest concentrations:
    mask = (amt_uM == amt_uM_range[i-3]) | (amt_uM == amt_uM_range[i-2]) | (amt_uM == amt_uM_range[i-1]) | (amt_uM == amt_uM_range[i]) | (amt_uM == amt_uM_range[i+1])
    amt_uM_tmp = amt_uM[mask]
    rsp_ratio_tmp = rsp_ratio[mask]

    piece_wise_fit[key_extra_low] = pick_best_fun_l1(amt_uM_tmp, rsp_ratio_tmp)


    ### Smoothen between the lower extrapolation and the intrapolation ###
    ub = rsp_ratio_range[len(amt_uM_range)-2] + smo_frac*rsp_ratio_range[len(amt_uM_range)-2]
    lb = rsp_ratio_range[len(amt_uM_range)-2] - smo_frac*rsp_ratio_range[len(amt_uM_range)-2]
    key_between_intra_low = (ub, lb)
    
    y_upper = piece_wise_fit[key_intra](ub)
    y_lower = piece_wise_fit[key_extra_low](lb)

    beta_lower = (y_upper - y_lower) / (ub - lb)
    if beta_lower < 0:
        print('Lower smoothening gave negative slope!!! Maybe adjust smo_frac.')
    alpha_lower = y_upper - beta_lower*ub
    piece_wise_fit[key_between_intra_low] = lambda x: alpha_lower + beta_lower*x

    
    if debug is True:
        plot_cal_curve(df, piece_wise_fit)

    return(piece_wise_fit)

In [4]:
def impute_conc(piece_wise_fit_metab, response_ratio):
    '''
    This function imputes the concentration from a response ratio.
    '''
    response_ratio_range = np.array(list(piece_wise_fit_metab.keys()))
    mask_range = [response_ratio >= min_v and response_ratio <= max_v for max_v, min_v in response_ratio_range]
    k = tuple(response_ratio_range[mask_range][0])
    return(piece_wise_fit_metab[k](response_ratio))

In [5]:
def plot_cal_curve(df, piece_wise_fit_metab):
    plt.figure(figsize=(5, 4))

    #plt.scatter(rr_mes_metab, imp_conc_mes_metab, color='red', marker='x')
    ax = sns.scatterplot(x="Response Ratio", y="Theoretical Amt", data=df)
    ax.set(xscale="log", yscale="log")
    ax.set_title('{}'.format(metab))

    for k in piece_wise_fit_metab.keys():
        ub, lb = k
        if ub == float('inf'):
            ub = 3*lb
        if lb == 0:
            lb = ub * 1e-1
        x = np.linspace(lb, ub, 1000)
        y = piece_wise_fit_metab[k](x)
        plt.plot(x, y, linewidth=1, color='black')


In [6]:
def plot_cal_piece_wise_fit(metab, df, piece_wise_fit_metab, save=False):
    if False: # Plot measured points on linear scale
        plt.figure(figsize=(5, 4))
        ax = sns.scatterplot(rr_mes_metab, imp_conc_mes_metab, color='red', marker='x')
        ax.set_title('Lin. mes. {}'.format(metab))
        plt.show()
    
    if True: # Plot both measured and calibration points on log-log scale
        if save:
            fig = plt.figure(figsize=(10, 8))
            plt.tight_layout()
        else:
            plt.figure(figsize=(5, 4))

        ax = sns.scatterplot(x="Response Ratio", y="Theoretical Amt", data=df)
        ax.set(xscale="log", yscale="log")
        ax.set_title('{}'.format(metab))

        for k in piece_wise_fit_metab.keys():
            ub, lb = k
            if ub == float('inf'):
                ub = 3*lb
            if lb == 0:
                lb = ub * 1e-1
            x = np.linspace(lb, ub, 1000)
            y = piece_wise_fit_metab[k](x)
            plt.plot(x, y, linewidth=1, color='black')

        plt.tight_layout()
        if save:
            return(fig)
        else:
            plt.show()

In [7]:
### Read calibration data

### Replace all N/F with 0 before start ###
esheet_dict_cal = pd.read_excel('AA_quant_oct20.xlsx', sheet_name=None)

metab_dict_cal = dict()
metab_names_cal = list()
for k in esheet_dict_cal.keys():
    if 'U-13C' not in k:
        metab_names_cal.append(k)
        # Remove non calibration standards and zero peaks:
        df_tmp = copy.deepcopy(esheet_dict_cal[k])
        mask_cal = df_tmp['Theoretical Amt'].values > 0
        metab_dict_cal[k] = df_tmp.loc[mask_cal]

In [8]:
### Fit calibration curve ###
piece_wise_fit = dict()
for metab in metab_names_cal:
    amt_uM = metab_dict_cal[metab]['Theoretical Amt'].values
    rsp_ratio = metab_dict_cal[metab]['Response Ratio'].values
    df = metab_dict_cal[metab]
    
    amt_uM_range = sorted(np.unique(amt_uM), reverse=True)
    rsp_ratio_range = [np.average(rsp_ratio[(amt_uM == r)]) for r in amt_uM_range]

    amt_uM_range.append(0)
    amt_uM_range.insert(0, float('inf'))
    rsp_ratio_range.append(0)
    rsp_ratio_range.insert(0, float('inf'))
    
    piece_wise_fit[metab] = make_piece_wise_fit(amt_uM_range, rsp_ratio_range, df, debug=False)


<ipython-input-3-def85428caff>:68: RuntimeWarning: divide by zero encountered in double_scalars
  beta_lower = (y_upper - y_lower) / (ub - lb)
<ipython-input-3-def85428caff>:71: RuntimeWarning: invalid value encountered in double_scalars
  alpha_lower = y_upper - beta_lower*ub


In [9]:
### Plot measured values on calibration curve ###
save_pdf = True
if save_pdf:
    pdf = matplotlib.backends.backend_pdf.PdfPages("calibration_curves.pdf")
    for metab in metab_names_cal:
        fig = plot_cal_piece_wise_fit(metab, metab_dict_cal[metab], piece_wise_fit[metab], save=save_pdf)
        pdf.savefig(fig)
        plt.close(fig)
    pdf.close()
else:
    for metab in metab_names_cal:
        plot_cal_piece_wise_fit(metab, metab_dict_cal[metab], piece_wise_fit[metab], save=save_pdf)

<ipython-input-3-def85428caff>:72: RuntimeWarning: invalid value encountered in multiply
  piece_wise_fit[key_between_intra_low] = lambda x: alpha_lower + beta_lower*x


In [10]:
### Read measurements

### Replace all N/F with 0 before start ###
esheet_dict_mes = pd.read_excel('Asp-uptake-flux.xlsx', sheet_name=None)

metab_dict_mes = dict()
metab_names_mes = list()
for k in esheet_dict_mes.keys():
    if 'U-13C' not in k:
        metab_names_mes.append(k)
        # Remove peaks with no internal standard:
        df_tmp = copy.deepcopy(esheet_dict_mes[k])
        mask_zero = df_tmp['ISTD Response'].values > 0
        # Remove blank:
        metab_dict_mes[k] = df_tmp.loc[mask_zero]
        metab_dict_mes[k]['Sample_name'] = [fn.split('_')[-1] for fn in metab_dict_mes[k]['Filename']]
        
        # CAA was reconstituted in 400uL instead of 500uL
        # CAA peak will then be 5/4 times what it should be
        ## RR will then be 4/5 of what it should be
        # Correction factor: 5/4
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Response Ratio'] * 5/4

In [11]:
metab_dict_mes['Aspartate neg']

,Compound,RT,Type,Filename,Area,Formula,Response Ratio,ISTD Response,Flag Details,Sample_name
0,Aspartate neg,10.29,Target Compound,KD032321_032621_AF1,827448091,C4H7NO4,1.03875,995998015,NaN,AF1
1,Aspartate neg,10.29,Target Compound,KD032321_032621_AF2,826060429,C4H7NO4,1.02750,1004896027,NaN,AF2
2,Aspartate neg,10.29,Target Compound,KD032321_032621_AF3,623081444,C4H7NO4,0.82750,941357197,NaN,AF3
3,Aspartate neg,10.29,Target Compound,KD032321_032621_AF4,690726936,C4H7NO4,0.84500,1021936456,NaN,AF4
4,Aspartate neg,10.29,Target Compound,KD032321_032621_AF5,650788605,C4H7NO4,0.84625,960606804,NaN,AF5
5,Aspartate neg,10.29,Target Compound,KD032321_032621_AF6,643634464,C4H7NO4,0.82250,977751892,NaN,AF6
6,Aspartate neg,10.29,Target Compound,KD032321_032621_AF7,620829252,C4H7NO4,0.81375,954063224,NaN,AF7
7,Aspartate neg,10.29,Target Compound,KD032321_032621_AF8,721924384,C4H7NO4,0.85625,1053217814,NaN,AF8
8,Aspartate neg,10.29,Target Compound,KD032321_032621_AF9,600891347,C4H7NO4,0.69875,1074584567,NaN,AF9
9,Aspartate neg,10.29,Target Compound,KD032321_032621_AF10,616530646,C4H7NO4,0.69000,1116114085,NaN,AF10


In [12]:
### Impute concentration and add to metabolite dataframe ###
rr_mes = dict()
imp_conc_mes = dict()
for metab in metab_names_mes:
    metab_split = metab.split()
    if len(metab_split) == 3:
        metab_no_iso = ' '.join([metab_split[0], metab_split[-1]])
    elif len(metab_split) == 2:
        metab_no_iso = metab
    else:
        raise Exception('{} not recognized metabolite name format'.format(metab))
    conc_list = list()
    for rr in metab_dict_mes[metab]['Response Ratio'].values:
        conc = impute_conc(piece_wise_fit[metab_no_iso], rr)
        conc_list.append(conc)
    metab_dict_mes[metab]['imputed_sample_conc'] = conc_list

    if metab_no_iso in rr_mes:
        rr_mes[metab_no_iso].extend(list(metab_dict_mes[metab]['Response Ratio'].values))
        imp_conc_mes[metab_no_iso].extend(list(metab_dict_mes[metab]['imputed_sample_conc'].values))
    else:
        rr_mes[metab_no_iso] = list(metab_dict_mes[metab]['Response Ratio'].values)
        imp_conc_mes[metab_no_iso] = list(metab_dict_mes[metab]['imputed_sample_conc'].values)

In [13]:
dil_fact_df = pd.read_csv('Asp-uptake-flux_meta.csv', sep=',')
dil_fact_dict = {k:v for k, v in zip(dil_fact_df['Sample_name'].values, dil_fact_df['dilution_factor'].values)}
vol_hour_dict = {k:v for k, v in zip(dil_fact_df['Sample_name'].values, dil_fact_df['vol_hour'].values)}
media_vol_dict = {k:v for k, v in zip(dil_fact_df['Sample_name'].values, dil_fact_df['Media_vol'].values)}
time_dict = {k:v for k, v in zip(dil_fact_df['Sample_name'].values, dil_fact_df['Time'].values)}

In [14]:
for metab in metab_names_mes:
    dil_fact = [dil_fact_dict[sn] for sn in metab_dict_mes[metab]['Sample_name']]
    vol_hour = [vol_hour_dict[sn] for sn in metab_dict_mes[metab]['Sample_name']]
    media_vol = [media_vol_dict[sn] for sn in metab_dict_mes[metab]['Sample_name']]
    time = [time_dict[sn] for sn in metab_dict_mes[metab]['Sample_name']]
    metab_dict_mes[metab]['dilution_factor'] = dil_fact
    metab_dict_mes[metab]['vol_hour'] = vol_hour
    metab_dict_mes[metab]['Media_vol'] = media_vol
    metab_dict_mes[metab]['Time'] = time
    metab_dict_mes[metab]['undil_sample_conc'] = metab_dict_mes[metab]['imputed_sample_conc'] / metab_dict_mes[metab]['dilution_factor']

In [15]:
### Add flux:
for metab in metab_names_mes:
    metab_dict_mes[metab]['fmoles'] = metab_dict_mes[metab]['undil_sample_conc'] * metab_dict_mes[metab]['Media_vol']
    
    blank_fmoles = metab_dict_mes[metab][metab_dict_mes[metab]['Time'] == 0]['fmoles'].mean()
    metab_dict_mes[metab]['nmoles_depleted'] = (blank_fmoles - metab_dict_mes[metab]['fmoles']) * 1e-3
    metab_dict_mes[metab]['Influx'] = metab_dict_mes[metab]['nmoles_depleted'] / metab_dict_mes[metab]['vol_hour'] * 1e-6


In [16]:
col_names = ['Samples_name'] + metab_names_mes
data0 = np.zeros((len(metab_dict_mes['Aspartate neg']), len(col_names)))
flux_df = pd.DataFrame(data0, columns=col_names)

In [17]:
flux_df['Samples_name'] = metab_dict_mes['Aspartate neg']['Sample_name'].values
for metab in metab_names_mes:
    flux_df[metab] = metab_dict_mes[metab]['Influx'].values

In [18]:
flux_df

,Samples_name,Alanine pos,Arginine pos,Asparagine pos,Aspartate neg,Glutamate neg,Glutamine pos,Glycine neg,Isoleucine pos,Leucine pos,Lysine pos,Methionine pos,Phenylalanine pos,Proline pos,Serine neg,Threonine neg,Tryptophan pos,Tyrosine pos,Valine pos
0,AF1,-inf,-inf,inf,-inf,NaN,-inf,inf,-inf,-inf,inf,-inf,-inf,NaN,inf,-inf,-inf,-inf,-inf
1,AF2,inf,inf,-inf,inf,NaN,inf,-inf,inf,inf,-inf,inf,inf,NaN,-inf,inf,inf,inf,inf
2,AF3,-0.231885,1.486352,-0.914361,4.241091,-7.387576,30.805739,0.467457,3.122836,3.398752,1.836688,0.581675,1.084094,-0.299336,3.745368,1.985481,0.309932,1.133651,2.692885
3,AF4,-0.288329,1.441158,-1.247871,4.601611,-8.246565,31.992951,-0.081917,3.261240,3.664018,2.105417,0.598870,1.052207,-0.336374,3.969409,2.019293,0.296474,0.817991,2.578928
4,AF5,-0.265694,1.945098,-0.994957,4.841581,-7.940382,35.951827,0.672795,2.511717,5.668190,2.715456,0.797660,1.388293,-0.324069,4.078427,3.056043,0.358420,1.310337,3.134047
5,AF6,-0.296989,1.851992,-0.993173,4.868082,-7.777196,35.189383,0.347509,3.446391,3.609576,2.879848,0.713595,1.234882,-0.315980,4.133065,2.480552,0.329712,1.360567,2.740514
6,AF7,-0.246400,1.874578,-0.974513,5.008025,-7.482871,34.488582,0.640250,2.836573,5.517394,2.927438,0.763195,1.402447,-0.300468,4.443762,2.525645,0.423718,1.402764,3.196791
7,AF8,-0.317173,1.639292,-1.132687,4.952783,-8.841801,36.089951,0.214532,2.074483,5.559161,2.424098,0.650738,1.209858,-0.349237,4.167004,2.354393,0.317570,1.215106,2.602967
8,AF9,-0.319100,1.198410,-1.078579,4.084392,-5.683267,24.851151,0.174117,1.797230,3.477135,1.801452,0.499700,0.807304,-0.263861,3.827497,1.225002,0.252920,0.858658,1.682629
9,AF10,-0.322696,1.574386,-1.144198,4.822041,-6.063331,30.257589,0.394139,2.633374,4.384960,2.514221,0.700533,1.234515,-0.271972,4.110286,2.288322,0.340934,1.265994,2.860842


In [19]:
metab_dict_mes['Glycine neg']

,Compound,RT,Type,Filename,Area,Formula,Response Ratio,ISTD Response,Flag Details,Sample_name,imputed_sample_conc,dilution_factor,vol_hour,Media_vol,Time,undil_sample_conc,fmoles,nmoles_depleted,Influx
0,Glycine neg,10.2,Target Compound,KD032321_032621_AF1,33100077,C2H5NO2,0.55625,74312176,NaN,AF1,30.722100,0.1,0.000000,2990.000000,0,307.221002,918590.795802,1.023927,inf
1,Glycine neg,10.2,Target Compound,KD032321_032621_AF2,35244114,C2H5NO2,0.55750,78948765,NaN,AF2,30.790590,0.1,0.000000,2990.000000,0,307.905903,920638.650174,-1.023927,-inf
2,Glycine neg,10.2,Target Compound,KD032321_032621_AF3,33400649,C2H5NO2,0.55750,74934971,NaN,AF3,30.790590,0.1,0.000091,2848.064516,23,307.905903,876935.876796,42.678846,0.467457
3,Glycine neg,10.2,Target Compound,KD032321_032621_AF4,33261280,C2H5NO2,0.59375,69985478,NaN,AF4,32.776804,0.1,0.000081,2825.806452,23,327.768036,926209.030265,-6.594307,-0.081917
4,Glycine neg,10.2,Target Compound,KD032321_032621_AF5,33407425,C2H5NO2,0.56000,74568865,NaN,AF5,30.927571,0.1,0.000078,2803.548387,23,309.275705,867069.404800,52.545318,0.672795
5,Glycine neg,10.2,Target Compound,KD032321_032621_AF6,31144012,C2H5NO2,0.57500,67665275,NaN,AF6,31.749452,0.1,0.000085,2803.548387,23,317.494519,890111.246231,29.503477,0.347509
6,Glycine neg,10.2,Target Compound,KD032321_032621_AF7,30291912,C2H5NO2,0.55750,67990703,NaN,AF7,30.790590,0.1,0.000084,2810.967742,23,307.905903,865513.561096,54.101162,0.640250
7,Glycine neg,10.2,Target Compound,KD032321_032621_AF8,32611726,C2H5NO2,0.58250,69929592,NaN,AF8,32.160393,0.1,0.000073,2810.967742,23,321.603926,904018.260640,15.596462,0.214532
8,Glycine neg,10.2,Target Compound,KD032321_032621_AF9,32464499,C2H5NO2,0.57250,70809654,NaN,AF9,31.612472,0.1,0.000143,2830.000000,31,316.124717,894632.947989,24.981775,0.174117
9,Glycine neg,10.2,Target Compound,KD032321_032621_AF10,30207555,C2H5NO2,0.56250,67064150,NaN,AF10,31.064551,0.1,0.000126,2800.000000,31,310.645508,869807.421225,49.807302,0.394139


In [19]:
impute_conc(piece_wise_fit['Glutamine pos'], 1)

61.11256466951882

In [21]:
flux_df.to_excel('influx.xlsx')